In [1]:
import json
import re
import sentencepiece as spm
import modeling
import numpy as np
import json
import tensorflow as tf

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
from prepro_utils import preprocess_text, encode_ids, encode_pieces

sp_model = spm.SentencePieceProcessor()
sp_model.Load('albert-large/sp10m.cased.v6.model')

with open('albert-large/sp10m.cased.v6.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [4]:
import optimization
import tokenization
import modeling
import numpy as np
import json
import tensorflow as tf
import itertools
from unidecode import unidecode
import re

In [5]:
ALBERT_INIT_CHKPNT = 'albert-large/model.ckpt'
ALBERT_CONFIG = 'albert-large/albert_config_large.json'

In [6]:
import glob

files = glob.glob('../Malaya-Dataset/emotion/translated*')
files

['../Malaya-Dataset/emotion/translated-love',
 '../Malaya-Dataset/emotion/translated-happy',
 '../Malaya-Dataset/emotion/translated-fear',
 '../Malaya-Dataset/emotion/translated-sadness',
 '../Malaya-Dataset/emotion/translated-surprise',
 '../Malaya-Dataset/emotion/translated-anger']

In [7]:
import re
from unidecode import unidecode

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [8]:
texts, labels = [], []

for file in files:
    label = file.split('translated-')[1]
    with open(file) as fopen:
        t = fopen.read().split('\n')[:-1]
    t = [cleaning(s) for s in t]
    t = list(filter(None, t))
    texts.extend(t)
    print(label, len(t))
    labels.extend([label] * len(t))

love 15231
happy 19586
fear 19057
sadness 16052
surprise 9711
anger 18872


In [9]:
files = glob.glob('../Malaya-Dataset/emotion/*malaysia.json')
files

['../Malaya-Dataset/emotion/surprise-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/happy-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/love-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/fear-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/anger-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/sadness-twitter-malaysia.json']

In [10]:
import json

for file in files:
    label = file.split('/')[-1].split('-')[0]
    with open(file) as fopen:
        t = json.load(fopen)
    t = [cleaning(s) for s in t]
    t = list(filter(None, t))
    texts.extend(t)
    print(label, len(t))
    labels.extend([label] * len(t))
    
len(texts), len(labels)

surprise 37778
happy 48924
love 59242
fear 18895
anger 51745
sadness 79233


(394326, 394326)

In [11]:
np.unique(labels, return_counts = True)

(array(['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
       dtype='<U8'), array([70617, 37952, 68510, 74473, 95285, 47489]))

In [12]:
from tqdm import tqdm

input_ids = []

for text in tqdm(texts):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["<cls>"] + tokens_a + ["<sep>"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    
    input_ids.append(input_id)

100%|██████████| 394326/394326 [00:29<00:00, 13210.14it/s]


In [13]:
albert_config = modeling.BertConfig.from_json_file(ALBERT_CONFIG)

In [14]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [15]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=albert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [16]:
dimension_output = 6
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ALBERT_INIT_CHKPNT)


embedding_lookup_factorized. factorized embedding parameterization is used.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from albert-large/model.ckpt


In [17]:
from sklearn.preprocessing import LabelEncoder

labels = LabelEncoder().fit_transform(labels)
labels = labels.tolist()

In [18]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y = train_test_split(
    input_ids, labels, test_size = 0.2
)

In [19]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [20]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/5258 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.854652
time taken: 3442.836123228073
epoch: 0, training loss: 0.538976, training acc: 0.773412, valid loss: 0.314000, valid acc: 0.854652



train minibatch loop:   0%|          | 0/5258 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.854652, current acc: 0.864517
time taken: 3439.907650947571
epoch: 1, training loss: 0.285511, training acc: 0.861869, valid loss: 0.275997, valid acc: 0.864517



train minibatch loop:  64%|██████▎   | 3346/5258 [33:33<18:14,  1.75it/s, accuracy=0.867, cost=0.22]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:   0%|          | 0/5258 [00:00<?, ?it/s]

epoch: 2, pass acc: 0.864517, current acc: 0.868962
time taken: 3437.9581015110016
epoch: 2, training loss: 0.254029, training acc: 0.869692, valid loss: 0.261888, valid acc: 0.868962



train minibatch loop:  16%|█▌        | 841/5258 [08:32<43:03,  1.71it/s, accuracy=0.85, cost=0.223]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  94%|█████████▎| 4922/5258 [49:26<03:02,  1.84it/s, accuracy=0.95, cost=0.191]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  72%|███████▏  | 3760/5258 [37:43<14:17,  1.75it/s, accuracy=0.867, cost=0.206] IOPub message rate exceeded.
The notebook se

time taken: 3437.525307416916
epoch: 4, training loss: 0.225585, training acc: 0.878355, valid loss: 0.259390, valid acc: 0.867166



test minibatch loop: 100%|██████████| 1315/1315 [04:30<00:00,  4.87it/s, accuracy=0.885, cost=0.142] 

time taken: 3439.975503921509
epoch: 5, training loss: 0.213827, training acc: 0.881332, valid loss: 0.272522, valid acc: 0.868670

break epoch:6



In [21]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 1315/1315 [04:26<00:00,  4.94it/s]


In [22]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
        digits = 5
    )
)

              precision    recall  f1-score   support

       anger    0.92852   0.92546   0.92699     14247
        fear    0.87507   0.88394   0.87948      7591
       happy    0.94091   0.90678   0.92353     13505
        love    0.93509   0.94468   0.93986     15022
     sadness    0.79416   0.83778   0.81539     18987
    surprise    0.71425   0.65892   0.68547      9514

    accuracy                        0.86866     78866
   macro avg    0.86467   0.85959   0.86179     78866
weighted avg    0.86855   0.86866   0.86827     78866

